SISTEMA DE RECOMENCACION SPOTIFY

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import when
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder \
    .appName("CrearDataFrame") \
    .master("local") \
    .getOrCreate()

In [3]:
df = spark.read.csv("C:/Dropbox/CursoSpark/pandas_spotify.csv", header=True, sep=",")

In [4]:
df.show()

+---+----+--------------------+---+------+---+-------------------+--------------------+---+-------------------+---+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+---+-----------+----+
|_c0|   0|                   1|  2|     3|  4|                  5|                   6|  7|                  8|  9|                  10|                  11|                  12|                  13|                 14|                15| 16|         17|  18|
+---+----+--------------------+---+------+---+-------------------+--------------------+---+-------------------+---+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+---+-----------+----+
|  0| 539|    Someone Like You| 57|276146|  0| 0.4390000104904175|  0.2070000022649765|  1| -9.572999954223633|  1| 0.02969999983906746|  0.6079999804496765|                 0.0|  0.1860000044107437| 0.2639999985694885|1

In [5]:
df_top10_mio = spark.createDataFrame(df.filter(df['17'] == 'alt-rock').take(10))
df_top10_mio = df_top10_mio.drop('_c0')

In [22]:
df_top10_mio.show()

+----+--------------------+---+------+---+-------------------+------------------+---+-------------------+---+--------------------+--------------------+--------------------+-------------------+-------------------+------------------+---+--------+---+
|   0|                   1|  2|     3|  4|                  5|                 6|  7|                  8|  9|                  10|                  11|                  12|                 13|                 14|                15| 16|      17| 18|
+----+--------------------+---+------+---+-------------------+------------------+---+-------------------+---+--------------------+--------------------+--------------------+-------------------+-------------------+------------------+---+--------+---+
|2260|          Can't Stop| 82|269000|  0| 0.6179999709129333|0.9380000233650208|  9| -3.441999912261963|  1| 0.04560000076889992|0.017899999395012856|                 0.0|0.16699999570846558|              0.875| 91.45500183105469|  4|alt-rock|1.0|
|236

In [6]:
def listaDeCanciones(df_top10_mio):
    lista_canciones = []
    id_canciones = df_top10_mio.select('0')
    id_canciones = id_canciones.collect()
    for i in id_canciones:
        lista_canciones.append(int(i[0]))
        
    return lista_canciones

In [7]:
def idGenero(df):
    id_genero = df[['0','17']]
    id_genero = id_genero.collect()
    
    return id_genero

In [25]:
def getStimilarMusicID(lista_canciones, id_genero):
    similar_music_id = []
    for i, g in id_genero:
        if i not in lista_canciones and g == 'alt-rock':
            similar_music_id.append(i)
    similar_music_id = list(set(similar_music_id))

    return similar_music_id

In [31]:
def createDfCantidato(df, similar_music_id):
    df_candidato = df.filter(col('0').isin(similar_music_id))

    return df_candidato

In [10]:
def arrayxConvert(df_top10_mio):
    df_top10_mio_clean = df_top10_mio.drop('_c0','1','17').collect()
    arrayx = np.array(df_top10_mio_clean)

    return arrayx

In [11]:
def arrayyConvert(df_candidato):
    df_candidato_clean = df_candidato.drop('_c0','1','17').collect()
    arrayy = np.array(df_candidato_clean)

    return arrayy

In [12]:
def compute_cossim(arrayx, arrayy):
    scaler = StandardScaler()
    top_songs_scaled = scaler.fit_transform(arrayx)
    can_scaled = scaler.fit_transform(arrayy)

    top_songs_norm = np.sqrt((top_songs_scaled * top_songs_scaled).sum(axis=1))
    can_norm = np.sqrt((can_scaled * can_scaled)).sum(axis=1)

    n_top_songs = top_songs_scaled.shape[0]
    n_candidates = can_scaled.shape[0]
    top_songs = top_songs_scaled/top_songs_norm.reshape(n_top_songs,1)
    candidates = can_scaled/can_norm.reshape(n_candidates,1)

    cos_sim = linear_kernel(top_songs, candidates)

    return cos_sim


In [54]:
def content_based_filtering(pos, cos_sim, ncands, threshold= 0.8):
    filtered_indices = np.where(cos_sim[pos, :] >= threshold)[0]

    sorted_indices = filtered_indices[np.argsort(cos_sim[pos, filtered_indices])[::-1]]
    
    selected_candidates = sorted_indices[:ncands]

    return selected_candidates


In [95]:
def output_recomendaciones(cancion):
    cands = content_based_filtering(cancion, cos_sim, ncands=5, threshold=0.1)
    
    return cands[1::]

In [56]:
def obtener_id_canciones(cands, arrayy):
    lista_ids = []
    for i in cands:
        lista_ids.append(arrayy[i][0])
    
    return lista_ids

In [57]:
def obtrener_nombre_canciones(lista_ids,df):
    df_id_musica = df.select('0','1')
    nombre_canciones = []
    canciones = df_id_musica.filter(col('0').isin(lista_ids)).collect()
    for i in canciones:
        nombre_canciones.append(i[1])
    return nombre_canciones

In [58]:
lista_canciones = listaDeCanciones(df_top10_mio)

In [60]:
id_genero = idGenero(df)

In [62]:
similar_music_id = getStimilarMusicID(lista_canciones, id_genero)

In [64]:
df_candidato = createDfCantidato(df, similar_music_id)

In [66]:
arrayx = arrayxConvert(df_top10_mio)

In [68]:
arrayy = arrayyConvert(df_candidato)

In [70]:
cos_sim = compute_cossim(arrayx, arrayy)

In [110]:
cos_sim.max()

0.31957840356329825

In [91]:
selected_candidates = content_based_filtering(1, cos_sim, ncands=5, threshold=0.1)


In [96]:
cands = output_recomendaciones(1)

In [98]:
lista_ids = obtener_id_canciones(cands, arrayy)

In [100]:
nombre_canciones = obtrener_nombre_canciones(lista_ids,df)

In [112]:
cancion = int(input("Cancion: "))
while True:
    if cancion <= 0:
        break
    else:
        selected_candidates = content_based_filtering(cancion, cos_sim, ncands=5, threshold=0.3)
        cands = output_recomendaciones(cancion)
        lista_ids = obtener_id_canciones(cands, arrayy)
        nombre_canciones = obtrener_nombre_canciones(lista_ids,df)
        print("En base a su top 10 canciones escuchadas, se recomiendan 4 canciones para cada una del top 10.")
        print("")
        for i in nombre_canciones:
            print("Cancion: ", str(i))
        cancion = int(input('Ingrese otro tema: '))

En base a su top 10 canciones escuchadas, se recomiendan 4 canciones para cada una del top 10.

Cancion:  Feel Invincible
Cancion:  Choke
Cancion:  Living Dead Girl
Cancion:  When You Were Young
En base a su top 10 canciones escuchadas, se recomiendan 4 canciones para cada una del top 10.

Cancion:  Heads Will Roll - A-Trak Remix
Cancion:  Heads Will Roll - A-Trak Remix
Cancion:  Get Innocuous! - Soulwax Remix
Cancion:  Heads Will Roll - A-Trak Remix


ValueError: invalid literal for int() with base 10: 'q'